# Run processing

Augment your data with processing algorithms.

- <a href="#step-1">1. Set up the notebook</a>
- <a href="#step-2">2. Select a process</a>
- <a href="#step-3">3. Prepare input parameters</a>
- <a href="#step-4">4. Determine the cost</a>
- <a href="#step-5">5. Run a process</a>
- <a href="#step-6">6. Monitor a job</a>
- <a href="#step-7">7. Download processing results</a>

## <a id="step-1"></a> 1. Set up the notebook

### 1. Install prerequisites

In [1]:
!pip install up42-py

import up42, os
from up42 import processing_templates

### 2. Authenticate

Create a `credentials.json` file in a directory named `.up42` under your home directory by running the cell below. The path to the file will be `~/.up42/credentials.json`.

In [2]:
# Define the directory path
up42_directory = os.path.expanduser("~/.up42")

# Create the directory if it doesn't exist
if not os.path.exists(up42_directory):
    os.makedirs(up42_directory)

# Specify the file path
credentials_file_path = os.path.join(up42_directory, "credentials.json")

# Check if the file already exists before creating it
if not os.path.exists(credentials_file_path):
    # Create an empty credentials.json file
    with open(credentials_file_path, "w") as credentials_file:
        print(f"The file {credentials_file_path} has been created.")
        pass
else:
    print(f"The file {credentials_file_path} already exists.")

The file /Users/max.mustermann/.up42/credentials.json already exists.


1. Click the link above to the created file and paste the following code:
    ```json
    {
        "username": "<your-email-address>",
        "password": "<your-password>"
    }
    ```
2. Retrieve the email address and password used for logging into the console. Use them as values for username and password.
3. Save the `credentials.json` file.
4. Check that the authentication was successful as follows:

In [3]:
up42.authenticate(cfg_file=credentials_file_path)

2024-07-17 14:23:31,579 - Authentication with UP42 successful!


## <a id="step-2"></a> 2. Select a process

[Processes](https://docs.up42.com/processing/processes) are algorithms that you can apply to [STAC items](https://docs.up42.com/developers/api-stac#items) in your storage. There are two types of processes:

- [Enhancement](https://docs.up42.com/processing/processes/enhancement) processes that pre-process imagery to improve its visual quality.
- [Analytics](https://docs.up42.com/processing/processes/analytics) processes that extract insights from imagery.

[Retrieve the class name](https://github.com/up42/up42-py/blob/master/up42/processing_templates.py) of the process you want to run.

If you want to run the chosen process for the first time, review and accept its end-user license agreement (EULA) in one of the following ways:

- [Via the console](https://docs.up42.com/getting-started/account/eulas)
- [Via the API](https://docs.up42.com/developers/api-eulas)

If a EULA is updated, you need to re-accept it before creating your next job.

## <a id="step-3"></a> 3. Prepare input parameters

### 1. Select input STAC items

You need asset IDs of one or several input STAC items you will apply the selected process to. Retrieve asset IDs in one of the following ways:

- [Retrieve IDs of specific assets](https://github.com/up42/documentation-notebooks/blob/main/storage/sdk-storage-assets.ipynb)
- [Retrieve IDs of assets delivered as part of a specific order](https://github.com/up42/documentation-notebooks/blob/main/storage/sdk-order-assets.ipynb)

In [4]:
item_id = "68567134-27ad-7bd7-4b65-d61adb11fc78"
title = "Process_example_SDK"

# If you have selected a process with 1 input STAC item
stac_client = up42.stac_client()
stac_item = next(stac_client.get_items(item_id))


# If you have selected a process with 2 input STAC items
# stac_item1 = next(stac_client.get_items(item_id))
# stac_item2 = next(stac_client.get_items(another_item_id))

### 2. Create a job template

Create a template by specifying the values of the [input parameters](https://github.com/up42/up42-py/blob/master/up42/processing_templates.py) for the process you have selected.

In [5]:
# Processes have different input parameters, but most of them only require a job title and a STAC item as DetectionAircraftOI
template = processing_templates.DetectionAircraftOI(
    title=title,
    item=stac_item
)

# template = processing_templates.DetectionChangePleiadesHyperverge(
#    title=title,
#    items=[stac_item1, stac_item2]
# )

### 3. Validate your input

Check the validity of your job template.

In [6]:
if not template.is_valid:
    print(template.errors)

## <a id="step-4"></a> 4. Determine the cost

Check the cost of a valid job template.

In [7]:
acceptable_cost = 100 # In UP42 credits
assert template.cost <= acceptable_cost
template.cost

Cost(strategy='area', credits=21, size=5.039398810389698, unit='SQ_KM')

## <a id="step-5"></a> 5. Run a process

In [8]:
job = template.execute()
job

## <a id="step-6"></a> 6. Monitor a job

### Track job statuses

The `track` instance method allows you to track the job status until it changes to `CAPTURED` or `RELEASED`. It will query the job status every 60 seconds for 3 days.

In [9]:
job.track()

### Search for specific jobs

In [10]:
workspace_id = "a0d443a2-41e8-4995-8b54-a5cc4c448227" # Your workspace ID
process_filter = [ # List of processes to search among
    processing_templates.Pansharpening.process_id,
    processing_templates.DetectionChangeSpacept.process_id,
]

# Sort by credit amount, from the least to the most expensive
sort_filter = up42.JobSorting.credits.asc

jobs = up42.Job.all(
#    process_id=process_filter,
#    workspace_id=workspace_id,
#    sort_by=sort_filter,
#    min_duration=20, # In seconds
#    max_duration=1000, # In seconds
)
for job in jobs:
    print(job)

### Retrieve a job by its ID

In [11]:
queried_job = up42.Job.get(job.id)
queried_job

## <a id="step-6"></a> 7. Download processing results

In a successfully completed job, use its `collection_url` to get the resulting STAC collection.

In [12]:
if job.status == up42.JobStatus.CAPTURED:
    print(job.collection_url)

Alternatively, use `job.collection` to obtain the `pystac.Collection` object directly.

In [13]:
job.collection